#Regressão linear com muitas variáveis categóricas

##Importando bibliotecas e dataframe

In [84]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

In [12]:
df = pd.read_csv('/content/CarPrice_Assignment.csv')

In [16]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


##Visualizando quantidade de dados categóricos e numéricos

In [14]:
df.describe(include=object)

,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,enginetype,cylindernumber,fuelsystem
count,205,205,205,205,205,205,205,205,205,205
unique,147,2,2,2,5,3,2,7,7,8
top,toyota corona,gas,std,four,sedan,fwd,front,ohc,four,mpfi
freq,6,185,168,115,96,120,202,148,159,94


Podemos perceber que pela quantidade de valores únicos na variável CarName, ela não é categórica, é apenas uma variável do tipo object que provavelmente não terá relação direta com o preço.

In [118]:
df.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


##Separando variáveis categóricas das numéricas para dummieficação 

In [34]:
dfC = df.select_dtypes(include = object)

In [35]:
dfC

,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,enginetype,cylindernumber,fuelsystem
0,alfa-romero giulia,gas,std,two,convertible,rwd,front,dohc,four,mpfi
1,alfa-romero stelvio,gas,std,two,convertible,rwd,front,dohc,four,mpfi
2,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,ohcv,six,mpfi
3,audi 100 ls,gas,std,four,sedan,fwd,front,ohc,four,mpfi
4,audi 100ls,gas,std,four,sedan,4wd,front,ohc,five,mpfi
...,...,...,...,...,...,...,...,...,...,...
200,volvo 145e (sw),gas,std,four,sedan,rwd,front,ohc,four,mpfi
201,volvo 144ea,gas,turbo,four,sedan,rwd,front,ohc,four,mpfi
202,volvo 244dl,gas,std,four,sedan,rwd,front,ohcv,six,mpfi
203,volvo 246,diesel,turbo,four,sedan,rwd,front,ohc,six,idi


In [45]:
dfCat = pd.get_dummies(dfC, columns = ['fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem'])

Aqui ignoramos a variável CarName, pois como foi citado anteriormente, ela provavelmente não tem nenhuma relação com o preço final.

In [47]:
dfCat.head()

,CarName,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,...,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,alfa-romero giulia,0,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,alfa-romero stelvio,0,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,alfa-romero Quadrifoglio,0,1,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,audi 100 ls,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,audi 100ls,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


##Juntando os tipos das variáveis novamente para o dataframe final

In [43]:
dfNum = df.select_dtypes(include = 'number')

In [78]:
X = dfNum.merge(dfCat, how='right')

In [88]:
X.select_dtypes(include=object)

,CarName
0,alfa-romero giulia
1,alfa-romero stelvio
2,alfa-romero Quadrifoglio
3,audi 100 ls
4,audi 100ls
...,...
200,volvo 145e (sw)
201,volvo 144ea
202,volvo 244dl
203,volvo 246


Vejos aqui que esquecemos de dropar a variável CarName do dataframe, então droparemos agora.

In [90]:
X = X.drop(['CarName'], axis = 1)

##Separando a variável target dos dados

In [81]:
y = X['price']

In [89]:
X = X.drop(['price'], axis = 1)

KeyError: ignored

##Separação da massa de teste do modelo

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 199)

##Treinando o modelo

In [104]:
regLinear = LinearRegression().fit(X_train, y_train)

In [111]:
yLinear = regLinear.predict(X_test)

##Analisando resultados do modelo

In [117]:
# Calcular as métricas do modelo

print("MSE: {}".format(metrics.mean_absolute_error(y_test,yLinear)))
print("RMSE: {}".format(metrics.mean_squared_error(y_test,yLinear)))
print("R²: {}".format(metrics.r2_score(y_test,yLinear)))

MSE: 2319.710083359973
RMSE: 7304287.937498943
R²: 0.8902974252614856
